In [1]:
import reframed
import pandas as pd
import seaborn as sns
from pathlib import Path
from sklearn import metrics
import numpy as np
sns.set(style='ticks', font_scale=1)
import scipy.integrate
from statannotations.Annotator import Annotator
from matplotlib import pyplot as plt

# Estimate the reduction in growth rates caused by the release in the knockout mutants

In [2]:
label_mapping = {
    'aceE-M2': 'ΔaceE-M2',
    'aceE-M3': 'ΔaceE-M3',
    'aceE-M4': 'ΔaceE-M4',
    'WT': 'E. coli WT',
    'aceE': 'ΔaceE',
    'sucB-M5': 'ΔsucB-M5',
    'sucB-M6': 'ΔsucB-M6',
    'sucB-M7': 'ΔsucB-M7',
    'sucB': 'ΔsucB',
}
tab10_cmap = sns.color_palette("tab10", 10)

set2 = sns.color_palette("Set2", 8)
strains_cmap = {
    'ΔaceE': tab10_cmap[1],
    'ΔsucB': tab10_cmap[8],
    'E. coli WT': tab10_cmap[0],
    'ΔaceE-M2':set2[3],
    'ΔaceE-M3':set2[0],
    'ΔaceE-M4':set2[6],
    'ΔsucB-M5':set2[5],
    'ΔsucB-M6':set2[2],
    'ΔsucB-M7':set2[7],
}



# Get rates

In [6]:
git_folder = Path('../..')
figure_folder = git_folder / 'Figures' /'chemostat'


data_folder_1 = git_folder / 'data' / 'this_project' / '3_keio_strains_chemostat'
fn_exometabolome_nM_updated = data_folder_1 / '3I_exometabolome_rates.csv'
df_nM = pd.read_csv(fn_exometabolome_nM_updated)

# Get model

In [8]:
model = reframed.load_cbmodel('../../models/e_coli/momentiJO1366_modified.xml')
# Use the original iJO1366 model for getting info about gene names
modeli =reframed.load_cbmodel('../../models/e_coli/iJO1366.xml')

In [9]:
gene_name_to_bcode = {}
for gi, g in modeli.genes.items():
    # print(gi, gname)
    # break
    if g.name in ['aceE', 'sucB']:
        print(gi, g.name)
        gene_name_to_bcode[g.name] = gi

m_aceE = model.copy()
m_aceE.remove_gene('G_b0114')

m_sucB = model.copy()
m_sucB.remove_gene('G_b0727')

G_b0727 sucB
G_b0114 aceE


In [10]:
# assume uptake is the same as in the WT
gal_uptake = -4.30

## aceE

In [11]:
mean_rates = df_nM.groupby(['Strain', 'Metabolite'], as_index=False).agg({'Rate [mmol/gDW/h]': 'mean'})
mean_rate_dict = mean_rates.set_index(['Strain', 'Metabolite'])['Rate [mmol/gDW/h]'].to_dict()

In [12]:
aceE_max = reframed.FBA(m_aceE, constraints={'R_EX_glc__D_e': 0, 'R_EX_gal_e': gal_uptake})
aceE_lactate = reframed.FBA(m_aceE, constraints={'R_EX_glc__D_e': 0, 'R_EX_gal_e': gal_uptake,
                                    'R_EX_lac__L_e': mean_rate_dict['aceE', 'Lactate'],
                                    # 'R_EX_pyr_e': mean_rate_dict['aceE', 'Pyruvate'],
                                    # 'R_EX_val__L_e': mean_rate_dict['aceE', 'Valine'],
                                    # 'R_EX_f6p_e': mean_rate_dict['aceE', 'Fructose-6-phosphate'],
                                   }, 
             objective=m_aceE.biomass_reaction)
aceE_pyr = reframed.FBA(m_aceE, constraints={'R_EX_glc__D_e': 0, 'R_EX_gal_e': gal_uptake,
                                    'R_EX_pyr_e': mean_rate_dict['aceE', 'Pyruvate']
                                   })

aceE_lactate_m2 = reframed.FBA(m_aceE, constraints={'R_EX_glc__D_e': 0, 'R_EX_gal_e': gal_uptake,
                                    'R_EX_lac__L_e': mean_rate_dict['aceE-M2', 'Lactate'],
                                   }, )


aceE_lactate_m4 = reframed.FBA(m_aceE, constraints={'R_EX_glc__D_e': 0, 'R_EX_gal_e': gal_uptake,
                                    'R_EX_lac__L_e': mean_rate_dict['aceE-M4', 'Lactate'],
                                   }, )

aceE_pyr_m4 = reframed.FBA(m_aceE, constraints={'R_EX_glc__D_e': 0, 'R_EX_gal_e': gal_uptake,
                                    'R_EX_pyr_e': mean_rate_dict['aceE-M4', 'Pyruvate']
                                   })


Set parameter Username
Set parameter LicenseID to value 2586336
Academic license - for non-commercial use only - expires 2025-11-18


In [17]:
print(aceE_max.fobj)

0.4052663579531723


In [13]:
lactate_growth_rate_reduction = 100*(1-aceE_lactate.fobj/aceE_max.fobj)
lactate_growth_rate_reduction_m2 = 100*(1-aceE_lactate_m2.fobj/aceE_max.fobj)
lactate_growth_rate_reduction_m4 = 100*(1-aceE_lactate_m4.fobj/aceE_max.fobj)
pyruvate_growth_rate_reduction = 100*(1-aceE_pyr.fobj/aceE_max.fobj)
pyruvate_growth_rate_reduction_m4 = 100*(1-aceE_pyr_m4.fobj/aceE_max.fobj)
print(f'Growth rate reduction caused by lactate: {lactate_growth_rate_reduction:.2f}%')
print(f'Growth rate reduction caused by lactate M2: {lactate_growth_rate_reduction_m2:.2f}%')
print(f'Growth rate reduction caused by lactate M4: {lactate_growth_rate_reduction_m4:.2f}%')
print(f'Growth rate reduction caused by pyruvate: {pyruvate_growth_rate_reduction:.4f}%')
print(f'Growth rate reduction caused by pyruvate M4: {pyruvate_growth_rate_reduction_m4:.4f}%')

Growth rate reduction caused by lactate: 6.06%
Growth rate reduction caused by lactate M2: 4.22%
Growth rate reduction caused by lactate M4: 1.95%
Growth rate reduction caused by pyruvate: 0.0063%
Growth rate reduction caused by pyruvate M4: 34.3158%


0.40526635795317206


In [128]:
aceE_max.show_values('R_EX')

R_EX_ca2_e   -0.00210941
R_EX_cl_e    -0.00210941
R_EX_co2_e    9.1645
R_EX_cobalt2_e -1.01317e-05
R_EX_cu2_e   -0.000287334
R_EX_fe2_e   -0.00650898
R_EX_gal_e   -4.3
R_EX_h_e      3.72368
R_EX_h2o_e    19.8675
R_EX_k_e     -0.0791052
R_EX_mg2_e   -0.00351569
R_EX_mn2_e   -0.000280039
R_EX_mobd_e  -5.22794e-05
R_EX_nh4_e   -4.3772
R_EX_ni2_e   -0.000130901
R_EX_o2_e    -8.2997
R_EX_pi_e    -0.390932
R_EX_so4_e   -0.102211
R_EX_zn2_e   -0.000138196


# sucB

In [14]:
m_sucB.add_reaction_from_str('R_DM_icit_c: M_icit_c --> [0,100]')
m_sucB.add_reaction_from_str('R_DM_acon_C_c: M_acon_C_c --> [0,100]')

'R_DM_acon_C_c'

0.40526635795317206


In [15]:
sucB_max = reframed.FBA(m_sucB, constraints={'R_EX_glc__D_e': 0, 'R_EX_gal_e': gal_uptake})
sucB_citrate = reframed.FBA(m_sucB, constraints={'R_EX_glc__D_e': 0, 'R_EX_gal_e': gal_uptake,
                                    'R_EX_cit_e': mean_rate_dict['sucB', 'Citrate'],
                                    'R_DM_acon_C_c': mean_rate_dict['sucB', 'Cis-aconitate'],
                                    'R_DM_icit_c': mean_rate_dict['sucB', 'Isocitrate'],
                                   }, 
             objective=m_sucB.biomass_reaction)

In [20]:
print(sucB_max.fobj)

0.40526635795317206


In [16]:
print(f'Growth rate reduction caused by citrate: {100*(1-sucB_citrate.fobj/sucB_max.fobj):.4f}%')

Growth rate reduction caused by citrate: 0.1792%
